## In the first part of the code, the data is processed and let ready for the computations.


### First the data from WRDS is loaded and cleaned.

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("data_wrds.csv",engine="python")

data.index = data["date"]
del data["date"]

data = data.drop('29/10/2012')
data = data.drop('30/09/2017')
data = data.drop('23/04/2005')

data.index = pd.to_datetime(data.index,dayfirst=True)

### Return data is cleaned

In [2]:
returns = pd.DataFrame(index=data.index,columns=["RETURNS"])

for i in range(0,len(data["RET"])):

    if data["RET"][i] == "C":

        pass 

    elif data["RET"][i] == "B":

        pass        
    
    else:
        returns.iloc[i] = float(data["RET"][i])    

data["RETURNS"] = returns

,PERMNO,COMNAM,PERMCO,CUSIP,RET,BID,ASK,SHROUT,RETURNS
date,,,,,,,,,
1996-01-02,10057,ACME CLEVELAND CORP NEW,20020,00462610,0.000000,18.75000,19.00000,6313.0,0.0
1996-01-03,10057,ACME CLEVELAND CORP NEW,20020,00462610,0.020000,18.75000,19.25000,6313.0,0.02
1996-01-04,10057,ACME CLEVELAND CORP NEW,20020,00462610,-0.026144,NaN,NaN,6313.0,-0.026144
1996-01-05,10057,ACME CLEVELAND CORP NEW,20020,00462610,0.000000,18.37500,18.75000,6313.0,0.0
1996-01-08,10057,ACME CLEVELAND CORP NEW,20020,00462610,0.000000,18.37500,18.75000,6313.0,0.0
...,...,...,...,...,...,...,...,...,...
2021-12-27,93436,TESLA INC,53453,88160R10,0.025248,1093.81995,1093.83997,1004265.0,0.025248
2021-12-28,93436,TESLA INC,53453,88160R10,-0.005000,1088.68005,1089.19995,1004265.0,-0.005
2021-12-29,93436,TESLA INC,53453,88160R10,-0.002095,1085.56995,1085.88000,1004265.0,-0.002095


### A list for the name of every security is created, so every ticker is labelled for the last name its security had.

In [3]:
names = pd.DataFrame(index=data["PERMNO"].unique(),columns=["Name"])

for i in data["PERMNO"].unique():
    names.loc[i] = data[data["PERMNO"]==i]["COMNAM"].unique()[-1]

### The same is done for company names, so we only remain with companies. Also, ETFs and Trusts on the sample are removed.

In [4]:
pre_permco_list = list(data["PERMCO"].unique())

delete_permcos = [22100,29010,29548,29941,30421,32030,37493,39147,44072,45684,45874,46673,50699,50846,51187,52902,52996,53171,54681,54917,56624,57105,21148,21584,22043,41593,
46673,53120,57105]

permco_list = [permco for permco in pre_permco_list if permco not in delete_permcos]

names_company = pd.DataFrame(index=permco_list,columns=["Name"])

for i in permco_list:
    names_company.loc[i] = data[data["PERMCO"]==i]["COMNAM"].unique()[-1]
    


### A data frame for ask price, shares outstanding and returns data from every company is created

In [5]:
ask = pd.DataFrame(index=data.index.unique(),columns=data["PERMNO"].unique())

for i in ask.columns:

   ask[i] = data[data["PERMNO"]==i]["ASK"]

ask.columns = names["Name"]

ask = ask[list(names_company["Name"])]

ask = ask.loc[:,~ask.columns.duplicated()]


In [6]:
shares = pd.DataFrame(index=data.index.unique(),columns=data["PERMNO"].unique())

for i in shares.columns:

   shares[i] = data[data["PERMNO"]==i]["SHROUT"]

shares.columns = names["Name"]

shares = shares[list(names_company["Name"])]

shares = shares.loc[:,~shares.columns.duplicated()]

### million of shares outstanding 

shares = shares/1000000

In [ ]:
returns = pd.DataFrame(index=data.index.unique(),columns=data["PERMNO"].unique())

for i in returns.columns:

   returns[i] = data[data["PERMNO"]==i]["RETURNS"]

returns.columns = names["Name"]

returns = returns[list(names_company["Name"])]

returns = returns.loc[:,~returns.columns.duplicated()]

returns

### Market cap is calculated and the eligility of every company is determined

In [ ]:
market_cap = ask*shares

eom_market_cap = market_cap.resample("M").pad()

eligibility = pd.DataFrame(index=eom_market_cap.index,columns=eom_market_cap.columns)

for m in range(0,len(eom_market_cap)):

    for cap in range(0,len(eom_market_cap.iloc[m])):

        if eom_market_cap.iloc[m,cap] > 5:

            eligibility.iloc[m,cap] = 1

        else:
            eligibility.iloc[m,cap] = np.nan